Dataset is from Kaggle competition. You can download the train and test data from :

https://www.kaggle.com/quora/question-pairs-dataset

In [1]:
import pandas as pd
import numpy as np
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
pd.set_option('display.max_columns', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import xgboost as xgb
from sklearn.decomposition import TruncatedSVD
import re, math
from collections import Counter
WORD = re.compile(r'\w+')
import scipy
import ngram
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk import word_tokenize
stop_words = stopwords.words('english')
import pyemd
import _pickle as cPickle
import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pyemd')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def str_common_word(str1, str2):
    return sum(int(str2.find(word)>=0) for word in str1.split())
def cooccurrence_terms(lst1, lst2, join_str):
    terms = [""] * len(lst1) * len(lst2)
    cnt =  0
    for item1 in lst1:
        for item2 in lst2:
            terms[cnt] = item1 + join_str + item2
            cnt += 1
    res = " ".join(terms)
    return res

def get_position_list(target, obs):
    """
        Get the list of positions of obs in target
    """
    pos_of_obs_in_target = [0]
    if len(obs) != 0:
        pos_of_obs_in_target = [j for j,w in enumerate(obs, start=1) if w in target]
        if len(pos_of_obs_in_target) == 0:
            pos_of_obs_in_target = [0]
    return pos_of_obs_in_target

In [3]:
def cos_sim(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return(Counter(words))
def cs(s,t):
    vector1 = text_to_vector(s)
    vector2 = text_to_vector(t)
    return cos_sim(vector1, vector2)

In [4]:
def try_divide(x, y, val=0.0):
    """ 
        Try to divide two numbers
    """
    if y != 0.0:
        val = float(x) / y
    return val
def JaccardCoef(A, B):
    A, B = set(A), set(B)
    intersect = len(A.intersection(B))
    union = len(A.union(B))
    coef = try_divide(intersect, union)
    return coef

def DiceDist(A, B):
    A, B = set(A), set(B)
    intersect = len(A.intersection(B))
    union = len(A) + len(B)
    d = try_divide(2*intersect, union)
    return d

def compute_dist(A, B, dist="jaccard_coef"):
    if dist == "jaccard_coef":
        d = JaccardCoef(A, B)
    elif dist == "dice_dist":
        d = DiceDist(A, B)
    return d

In [5]:
data_path = "/Users/araghavan/Quora/"
train = pd.read_csv(data_path+"train.csv")
test = pd.read_csv(data_path+"test.csv")
train.shape,test.shape

((404290, 6), (2345796, 3))

In [6]:
train.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
test.head(5)

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [8]:
train.is_duplicate.value_counts()

0    255027
1    149263
Name: is_duplicate, dtype: int64

In [9]:
train.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       2
is_duplicate    0
dtype: int64

In [10]:
train[train['question2'].isnull()]

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0


In [11]:
train[train['id']==174364]

,id,qid1,qid2,question1,question2,is_duplicate
174364,174364,268748,268749,Do you enjoy your job? What do you do and why?,Do you enjoy your job?,1


In [12]:
train[train['id']==174363]

,id,qid1,qid2,question1,question2,is_duplicate
174363,174363,268746,268747,What percentage of PhD students fail their the...,Do students cheat on their PhD thesis?,0


In [13]:
train[train['id']==303951]

,id,qid1,qid2,question1,question2,is_duplicate
303951,303951,240830,427149,Does USPS print labels for you?,Now that C Regional boxes are discontinued wit...,0


In [14]:
train = train[~train['question2'].isnull()]

In [15]:
train.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [16]:
test.isnull().sum()

test_id      0
question1    2
question2    4
dtype: int64

In [17]:
test[test['question2'].isnull()]

,test_id,question1,question2
379205,379205,How I can learn android app development?,NaN
817520,817520,How real can learn android app development?,NaN
943911,943911,How app development?,NaN
1270024,1270024,How I can learn app development?,NaN


In [18]:
test[test['question1'].isnull()]

,test_id,question1,question2
1046690,1046690,NaN,How I what can learn android app development?
1461432,1461432,NaN,How distinct can learn android app development?


In [19]:
submission = pd.read_csv(data_path+"sample_submission.csv")
submission.head(5)

,test_id,is_duplicate
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [20]:
train = train.iloc[0:200,]
test = test.iloc[0:200,]

In [21]:
train.shape,test.shape

((200, 6), (200, 3))

In [22]:
train_test = pd.concat((train[['id','question1','question2']], test[['test_id','question1','question2']]), axis=0, ignore_index=True)

In [23]:
train.loc[1,'question1'].lower().split()

['what', 'is', 'the', 'story', 'of', 'kohinoor', '(koh-i-noor)', 'diamond?']

In [24]:
train_test['question1'] = train_test['question1'].apply(lambda x: x.lower())
train_test['question2'] = train_test['question2'].apply(lambda x: x.lower())

In [25]:
#normal features
train_test['question1_length'] = train_test['question1'].apply(lambda x: len(x))
train_test['question2_length'] = train_test['question2'].apply(lambda x: len(x))
train_test['length_diff'] = abs(train_test['question1_length'] - train_test['question2_length'])

train_test['question1_word_length'] = train_test['question1'].apply(lambda x: len(x.split(' ')))
train_test['question2_word_length'] = train_test['question2'].apply(lambda x: len(x.split(' ')))
train_test['word_length_diff'] = abs(train_test['question1_word_length'] - train_test['question2_word_length'])

train_test['len_char_question1'] = train_test.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
train_test['len_char_question2'] = train_test.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))


train_test['common_words'] = train_test.apply(lambda row:str_common_word(row['question1'],row['question2']),axis=1)
train_test['ratio_question1'] = train_test['common_words']/train_test['question1_word_length']
train_test['ratio_question2'] = train_test['common_words']/train_test['question2_word_length']

train_test['cosine_similarity'] = train_test.apply(lambda row:cs(row['question1'],row['question2']),axis=1)

print("generate unigram")
train_test["question1_unigram"] = train_test['question1'].apply(lambda x: ngram.getUnigram(x.split()))
train_test["question2_unigram"] = train_test['question2'].apply(lambda x: ngram.getUnigram(x.split()))
 
print("generate bigram")
join_str = "_"
train_test["question1_bigram"] = list(train_test.apply(lambda x: ngram.getBigram(x["question1"].split(), join_str), axis=1))
train_test["question2_bigram"] = list(train_test.apply(lambda x: ngram.getBigram(x["question2"].split(), join_str), axis=1))

print("generate trigram")
join_str = "_"
train_test["question1_trigram"] = list(train_test.apply(lambda x: ngram.getTrigram(x["question1"].split(), join_str), axis=1))
train_test["question2_trigram"] = list(train_test.apply(lambda x: ngram.getTrigram(x["question2"].split(), join_str), axis=1))
    
join_str = "X"
# question1 unigram
train_test["question1_unigram_question2_unigram"] = list(train_test.apply(lambda x: cooccurrence_terms(x["question1_unigram"], x["question2_unigram"], join_str), axis=1))

# question1 bigram
train_test["question1_bigram_question2_unigram"] = list(train_test.apply(lambda x: cooccurrence_terms(x["question1_bigram"], x["question2_unigram"], join_str), axis=1))
    
    
print("generate word counting features")
feat_names = ["question1", "question2"]
grams = ["unigram","bigram", "trigram"]
count_digit = lambda x: sum([1. for w in x if w.isdigit()])
for feat_name in feat_names:
    for gram in grams:
        ## word count
        train_test["count_of_%s_%s"%(feat_name,gram)] = list(train_test.apply(lambda x: len(x[feat_name+"_"+gram]), axis=1))
        train_test["count_of_unique_%s_%s"%(feat_name,gram)] = list(train_test.apply(lambda x: len(set(x[feat_name+"_"+gram])), axis=1))
        train_test["ratio_of_unique_%s_%s"%(feat_name,gram)] = train_test.apply(lambda x:try_divide(x["count_of_unique_%s_%s"%(feat_name,gram)], x["count_of_%s_%s"%(feat_name,gram)]),axis=1)
print("generate intersect word counting features")
#### unigram
obs_name = 'question1'
target_name = 'question2'
for gram in grams:
    ## question1
    train_test["count_of_%s_%s_in_%s"%(obs_name,gram,target_name)] = list(train_test.apply(lambda x: sum([1. for w in x[obs_name+"_"+gram] if w in set(x[target_name+"_"+gram])]), axis=1))
    train_test["ratio_of_%s_%s_in_%s"%(obs_name,gram,target_name)] = train_test.apply(lambda x:try_divide(x["count_of_%s_%s_in_%s"%(obs_name,gram,target_name)], x["count_of_%s_%s"%(obs_name,gram)]),axis=1)
    
print("generate intersect word position features")
for gram in grams:
    pos = list(train_test.apply(lambda x: get_position_list(x[target_name+"_"+gram], obs=x[obs_name+"_"+gram]), axis=1))
    ## stats feat on pos
    train_test["pos_of_%s_%s_in_%s_min" % (obs_name, gram, target_name)] = list(np.min(i) for i in pos)
    train_test["pos_of_%s_%s_in_%s_mean" % (obs_name, gram, target_name)] = list(np.mean(i) for i in pos)
    train_test["pos_of_%s_%s_in_%s_median" % (obs_name, gram, target_name)] = list(np.median(i) for i in pos)
    train_test["pos_of_%s_%s_in_%s_max" % (obs_name, gram, target_name)] = list(np.max(i) for i in pos)
    train_test["pos_of_%s_%s_in_%s_std" % (obs_name, gram, target_name)] = list(np.std(i) for i in pos)
    ## stats feat on normalized_pos
    train_test["normalized_pos_of_%s_%s_in_%s_min" % (obs_name, gram, target_name)] = train_test.apply(lambda x:try_divide(x["pos_of_%s_%s_in_%s_min" % (obs_name, gram, target_name)], x["count_of_%s_%s" % (obs_name, gram)]),axis=1)
    train_test["normalized_pos_of_%s_%s_in_%s_mean" % (obs_name, gram, target_name)] = train_test.apply(lambda x:try_divide(x["pos_of_%s_%s_in_%s_mean" % (obs_name, gram, target_name)], x["count_of_%s_%s" % (obs_name, gram)]),axis=1)
    train_test["normalized_pos_of_%s_%s_in_%s_median" % (obs_name, gram, target_name)] = train_test.apply(lambda x:try_divide(x["pos_of_%s_%s_in_%s_median" % (obs_name, gram, target_name)], x["count_of_%s_%s" % (obs_name, gram)]),axis=1)
    train_test["normalized_pos_of_%s_%s_in_%s_max" % (obs_name, gram, target_name)] = train_test.apply(lambda x:try_divide(x["pos_of_%s_%s_in_%s_max" % (obs_name, gram, target_name)], x["count_of_%s_%s" % (obs_name, gram)]),axis=1)
    train_test["normalized_pos_of_%s_%s_in_%s_std" % (obs_name, gram, target_name)] = train_test.apply(lambda x:try_divide(x["pos_of_%s_%s_in_%s_std" % (obs_name, gram, target_name)] , x["count_of_%s_%s" % (obs_name, gram)]),axis=1)
    
print("generate jaccard coef and dice dist for n-gram")
dists = ["jaccard_coef", "dice_dist"]
feat_names = ["question1", "question2"]
grams = ["unigram","bigram", "trigram"]
for dist in dists:
    for gram in grams:
        train_test["%s_of_%s_between_%s_%s"%(dist,gram,target_name,obs_name)] = \
        list(train_test.apply(lambda x: compute_dist(x[target_name+"_"+gram], x[obs_name+"_"+gram], dist), axis=1))



generate unigram
generate bigram
generate trigram
generate word counting features
generate intersect word counting features
generate intersect word position features
generate jaccard coef and dice dist for n-gram


In [26]:
train_test.head(5)

,id,question1,question2,test_id,question1_length,question2_length,length_diff,question1_word_length,question2_word_length,word_length_diff,len_char_question1,len_char_question2,common_words,ratio_question1,ratio_question2,cosine_similarity,question1_unigram,question2_unigram,question1_bigram,question2_bigram,question1_trigram,question2_trigram,question1_unigram_question2_unigram,question1_bigram_question2_unigram,count_of_question1_unigram,count_of_unique_question1_unigram,ratio_of_unique_question1_unigram,count_of_question1_bigram,count_of_unique_question1_bigram,ratio_of_unique_question1_bigram,count_of_question1_trigram,count_of_unique_question1_trigram,ratio_of_unique_question1_trigram,count_of_question2_unigram,count_of_unique_question2_unigram,ratio_of_unique_question2_unigram,count_of_question2_bigram,count_of_unique_question2_bigram,ratio_of_unique_question2_bigram,count_of_question2_trigram,count_of_unique_question2_trigram,ratio_of_unique_question2_trigram,count_of_question1_unigram_in_question2,ratio_of_question1_unigram_in_question2,count_of_question1_bigram_in_question2,ratio_of_question1_bigram_in_question2,count_of_question1_trigram_in_question2,ratio_of_question1_trigram_in_question2,pos_of_question1_unigram_in_question2_min,pos_of_question1_unigram_in_question2_mean,pos_of_question1_unigram_in_question2_median,pos_of_question1_unigram_in_question2_max,pos_of_question1_unigram_in_question2_std,normalized_pos_of_question1_unigram_in_question2_min,normalized_pos_of_question1_unigram_in_question2_mean,normalized_pos_of_question1_unigram_in_question2_median,normalized_pos_of_question1_unigram_in_question2_max,normalized_pos_of_question1_unigram_in_question2_std,pos_of_question1_bigram_in_question2_min,pos_of_question1_bigram_in_question2_mean,pos_of_question1_bigram_in_question2_median,pos_of_question1_bigram_in_question2_max,pos_of_question1_bigram_in_question2_std,normalized_pos_of_question1_bigram_in_question2_min,normalized_pos_of_question1_bigram_in_question2_mean,normalized_pos_of_question1_bigram_in_question2_median,normalized_pos_of_question1_bigram_in_question2_max,normalized_pos_of_question1_bigram_in_question2_std,pos_of_question1_trigram_in_question2_min,pos_of_question1_trigram_in_question2_mean,pos_of_question1_trigram_in_question2_median,pos_of_question1_trigram_in_question2_max,pos_of_question1_trigram_in_question2_std,normalized_pos_of_question1_trigram_in_question2_min,normalized_pos_of_question1_trigram_in_question2_mean,normalized_pos_of_question1_trigram_in_question2_median,normalized_pos_of_question1_trigram_in_question2_max,normalized_pos_of_question1_trigram_in_question2_std,jaccard_coef_of_unigram_between_question2_question1,jaccard_coef_of_bigram_between_question2_question1,jaccard_coef_of_trigram_between_question2_question1,dice_dist_of_unigram_between_question2_question1,dice_dist_of_bigram_between_question2_question1,dice_dist_of_trigram_between_question2_question1
0,0.0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,NaN,66,57,9,14,12,2,20,20,13,0.928571,1.083333,0.944911,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...","[what_is, is_the, the_step, step_by, by_step, ...","[what_is, is_the, the_step, step_by, by_step, ...","[what_is_the, is_the_step, the_step_by, step_b...","[what_is_the, is_the_step, the_step_by, step_b...",whatXwhat whatXis whatXthe whatXstep whatXby w...,what_isXwhat what_isXis what_isXthe what_isXst...,14,12,0.857143,13,13,1.0,12,12,1.0,12,11,0.916667,11,11,1.0,10,10,1.0,12.0,0.857143,10.0,0.769231,9.0,0.75,1,6.583333,6.5,13,3.593010,0.071429,0.470238,0.464286,0.928571,0.256644,1,5.5,5.5,10,2.872281,0.076923,0.423077,0.423077,0.769231,0.220945,1,5.0,5.0,9,2.581989,0.083333,0.416667,0.416667,0.75,0.215166,0.769231,0.714286,0.692308,0.869565,0.833333,0.818182
1,1.0,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...

In [27]:
#Fuzzy features
from fuzzywuzzy import fuzz
train_test['fuzz_ratio'] = train_test.apply(lambda x: fuzz.ratio(str(x['question1']), str(x['question2'])), axis=1)
train_test['fuzz_qratio'] = train_test.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
train_test['fuzz_WRatio'] = train_test.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
train_test['fuzz_partial_ratio'] = train_test.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
train_test['fuzz_partial_token_set_ratio'] = train_test.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
train_test['fuzz_partial_token_sort_ratio'] = train_test.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
train_test['fuzz_token_set_ratio'] = train_test.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
train_test['fuzz_token_sort_ratio'] = train_test.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)


In [28]:
train_test.head(5)

,id,question1,question2,test_id,question1_length,question2_length,length_diff,question1_word_length,question2_word_length,word_length_diff,len_char_question1,len_char_question2,common_words,ratio_question1,ratio_question2,cosine_similarity,question1_unigram,question2_unigram,question1_bigram,question2_bigram,question1_trigram,question2_trigram,question1_unigram_question2_unigram,question1_bigram_question2_unigram,count_of_question1_unigram,count_of_unique_question1_unigram,ratio_of_unique_question1_unigram,count_of_question1_bigram,count_of_unique_question1_bigram,ratio_of_unique_question1_bigram,count_of_question1_trigram,count_of_unique_question1_trigram,ratio_of_unique_question1_trigram,count_of_question2_unigram,count_of_unique_question2_unigram,ratio_of_unique_question2_unigram,count_of_question2_bigram,count_of_unique_question2_bigram,ratio_of_unique_question2_bigram,count_of_question2_trigram,count_of_unique_question2_trigram,ratio_of_unique_question2_trigram,count_of_question1_unigram_in_question2,ratio_of_question1_unigram_in_question2,count_of_question1_bigram_in_question2,ratio_of_question1_bigram_in_question2,count_of_question1_trigram_in_question2,ratio_of_question1_trigram_in_question2,pos_of_question1_unigram_in_question2_min,pos_of_question1_unigram_in_question2_mean,pos_of_question1_unigram_in_question2_median,pos_of_question1_unigram_in_question2_max,pos_of_question1_unigram_in_question2_std,normalized_pos_of_question1_unigram_in_question2_min,normalized_pos_of_question1_unigram_in_question2_mean,normalized_pos_of_question1_unigram_in_question2_median,normalized_pos_of_question1_unigram_in_question2_max,normalized_pos_of_question1_unigram_in_question2_std,pos_of_question1_bigram_in_question2_min,pos_of_question1_bigram_in_question2_mean,pos_of_question1_bigram_in_question2_median,pos_of_question1_bigram_in_question2_max,pos_of_question1_bigram_in_question2_std,normalized_pos_of_question1_bigram_in_question2_min,normalized_pos_of_question1_bigram_in_question2_mean,normalized_pos_of_question1_bigram_in_question2_median,normalized_pos_of_question1_bigram_in_question2_max,normalized_pos_of_question1_bigram_in_question2_std,pos_of_question1_trigram_in_question2_min,pos_of_question1_trigram_in_question2_mean,pos_of_question1_trigram_in_question2_median,pos_of_question1_trigram_in_question2_max,pos_of_question1_trigram_in_question2_std,normalized_pos_of_question1_trigram_in_question2_min,normalized_pos_of_question1_trigram_in_question2_mean,normalized_pos_of_question1_trigram_in_question2_median,normalized_pos_of_question1_trigram_in_question2_max,normalized_pos_of_question1_trigram_in_question2_std,jaccard_coef_of_unigram_between_question2_question1,jaccard_coef_of_bigram_between_question2_question1,jaccard_coef_of_trigram_between_question2_question1,dice_dist_of_unigram_between_question2_question1,dice_dist_of_bigram_between_question2_question1,dice_dist_of_trigram_between_question2_question1,fuzz_ratio,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio
0,0.0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,NaN,66,57,9,14,12,2,20,20,13,0.928571,1.083333,0.944911,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...","[what_is, is_the, the_step, step_by, by_step, ...","[what_is, is_the, the_step, step_by, by_step, ...","[what_is_the, is_the_step, the_step_by, step_b...","[what_is_the, is_the_step, the_step_by, step_b...",whatXwhat whatXis whatXthe whatXstep whatXby w...,what_isXwhat what_isXis what_isXthe what_isXst...,14,12,0.857143,13,13,1.0,12,12,1.0,12,11,0.916667,11,11,1.0,10,10,1.0,12.0,0.857143,10.0,0.769231,9.0,0.75,1,6.583333,6.5,13,3.593010,0.071429,0.470238,0.464286,0.928571,0.256644,1,5.5,5.5,10,2.872281,0.076923,0.423077,0.423077,0.769231,0.220945,1,5.0,5.0,9,2.581989,0.083333,0.416667,0.416667,0.75,0.215166,0.7

In [29]:
#w2v features

def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)


def norm_wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return norm_model.wmdistance(s1, s2)


def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

In [30]:
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/araghavan/Downloads/GoogleNews-vectors-negative300.bin.gz', binary=True)


In [32]:
model.syn0

array([[  1.12915039e-03,  -8.96453857e-04,   3.18527222e-04, ...,
         -1.56402588e-03,  -1.23023987e-04,  -8.63075256e-05],
       [  7.03125000e-02,   8.69140625e-02,   8.78906250e-02, ...,
         -4.76074219e-02,   1.44653320e-02,  -6.25000000e-02],
       [ -1.17797852e-02,  -4.73632812e-02,   4.46777344e-02, ...,
          7.12890625e-02,  -3.49121094e-02,   2.41699219e-02],
       ..., 
       [ -1.96533203e-02,  -9.08203125e-02,  -1.94091797e-02, ...,
         -1.63574219e-02,  -1.34277344e-02,   4.66308594e-02],
       [  3.27148438e-02,  -3.22265625e-02,   3.61328125e-02, ...,
         -8.85009766e-03,   2.69775391e-02,   1.90429688e-02],
       [  4.51660156e-02,  -4.51660156e-02,  -3.93676758e-03, ...,
          7.95898438e-02,   7.22656250e-02,   1.30004883e-02]], dtype=float32)

In [33]:
train_test['wmd'] = train_test.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)
size = 300

question1_vectors = np.zeros((train_test.shape[0], size))
error_count = 0

for i, q in tqdm(enumerate(train_test.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

question2_vectors  = np.zeros((train_test.shape[0], size))
for i, q in tqdm(enumerate(train_test.question2.values)):
    question2_vectors[i, :] = sent2vec(q)
    
cPickle.dump(question1_vectors, open('/Users/araghavan/Quora/q1_w2v.pkl', 'wb'), -1)
cPickle.dump(question2_vectors, open('/Users/araghavan/Quora/q2_w2v.pkl', 'wb'), -1)

254it [00:00, 2538.04it/s]/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:34: RuntimeWarning: invalid value encountered in double_scalars
400it [00:00, 2676.01it/s]
400it [00:00, 3003.73it/s]


In [34]:
##simialrity based on w2v features

train_test['cosine_distance_w2v'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

train_test['cityblock_distance_w2v'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

train_test['jaccard_distance_w2v'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

train_test['canberra_distance_w2v'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

train_test['euclidean_distance_w2v'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

train_test['minkowski_distance_w2v'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

train_test['braycurtis_distance_w2v'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

train_test['skew_q1_w2v'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
train_test['skew_q2_w2v'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
train_test['kur_q1_w2v'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
train_test['kur_q2_w2v'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

#train_test.to_csv('/Users/araghavan/Quora/quora_features.csv', index=False)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [35]:
train_test.head(5)

,id,question1,question2,test_id,question1_length,question2_length,length_diff,question1_word_length,question2_word_length,word_length_diff,len_char_question1,len_char_question2,common_words,ratio_question1,ratio_question2,cosine_similarity,question1_unigram,question2_unigram,question1_bigram,question2_bigram,question1_trigram,question2_trigram,question1_unigram_question2_unigram,question1_bigram_question2_unigram,count_of_question1_unigram,count_of_unique_question1_unigram,ratio_of_unique_question1_unigram,count_of_question1_bigram,count_of_unique_question1_bigram,ratio_of_unique_question1_bigram,count_of_question1_trigram,count_of_unique_question1_trigram,ratio_of_unique_question1_trigram,count_of_question2_unigram,count_of_unique_question2_unigram,ratio_of_unique_question2_unigram,count_of_question2_bigram,count_of_unique_question2_bigram,ratio_of_unique_question2_bigram,count_of_question2_trigram,count_of_unique_question2_trigram,ratio_of_unique_question2_trigram,count_of_question1_unigram_in_question2,ratio_of_question1_unigram_in_question2,count_of_question1_bigram_in_question2,ratio_of_question1_bigram_in_question2,count_of_question1_trigram_in_question2,ratio_of_question1_trigram_in_question2,pos_of_question1_unigram_in_question2_min,pos_of_question1_unigram_in_question2_mean,pos_of_question1_unigram_in_question2_median,pos_of_question1_unigram_in_question2_max,pos_of_question1_unigram_in_question2_std,normalized_pos_of_question1_unigram_in_question2_min,normalized_pos_of_question1_unigram_in_question2_mean,normalized_pos_of_question1_unigram_in_question2_median,normalized_pos_of_question1_unigram_in_question2_max,normalized_pos_of_question1_unigram_in_question2_std,pos_of_question1_bigram_in_question2_min,pos_of_question1_bigram_in_question2_mean,pos_of_question1_bigram_in_question2_median,pos_of_question1_bigram_in_question2_max,pos_of_question1_bigram_in_question2_std,normalized_pos_of_question1_bigram_in_question2_min,normalized_pos_of_question1_bigram_in_question2_mean,normalized_pos_of_question1_bigram_in_question2_median,normalized_pos_of_question1_bigram_in_question2_max,normalized_pos_of_question1_bigram_in_question2_std,pos_of_question1_trigram_in_question2_min,pos_of_question1_trigram_in_question2_mean,pos_of_question1_trigram_in_question2_median,pos_of_question1_trigram_in_question2_max,pos_of_question1_trigram_in_question2_std,normalized_pos_of_question1_trigram_in_question2_min,normalized_pos_of_question1_trigram_in_question2_mean,normalized_pos_of_question1_trigram_in_question2_median,normalized_pos_of_question1_trigram_in_question2_max,normalized_pos_of_question1_trigram_in_question2_std,jaccard_coef_of_unigram_between_question2_question1,jaccard_coef_of_bigram_between_question2_question1,jaccard_coef_of_trigram_between_question2_question1,dice_dist_of_unigram_between_question2_question1,dice_dist_of_bigram_between_question2_question1,dice_dist_of_trigram_between_question2_question1,fuzz_ratio,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio,wmd,cosine_distance_w2v,cityblock_distance_w2v,jaccard_distance_w2v,canberra_distance_w2v,euclidean_distance_w2v,minkowski_distance_w2v,braycurtis_distance_w2v,skew_q1_w2v,skew_q2_w2v,kur_q1_w2v,kur_q2_w2v
0,0.0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,NaN,66,57,9,14,12,2,20,20,13,0.928571,1.083333,0.944911,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...","[what_is, is_the, the_step, step_by, by_step, ...","[what_is, is_the, the_step, step_by, by_step, ...","[what_is_the, is_the_step, the_step_by, step_b...","[what_is_the, is_the_step, the_step_by, step_b...",whatXwhat whatXis whatXthe whatXstep whatXby w...,what_isXwhat what_isXis what_isXthe what_isXst...,14,12,0.857143,13,13,1.0,12,12,1.0,12,11,0.916667,11,11,1.0,10,10,1.0,12.0,0.857143,10.0,0.769231,9.0,0.7

In [36]:
##Tfidf vectors
vectorizer = CountVectorizer(ngram_range=(1, 1), min_df=1, max_df = 0.2)
all_vecs1 = vectorizer.fit(train_test["question1"])
all_vecs = vectorizer.fit(train_test["question2"])
vocabulary = all_vecs.vocabulary_
tfidf = TfidfVectorizer(vocabulary=vocabulary)
question1_tfidf_vecs = tfidf.fit_transform(train_test['question1'])
question2_tfidf_vecs = tfidf.fit_transform(train_test['question2'])



In [37]:
#tfidf vector features
train_test['sum_question1_tfidf'] = question1_tfidf_vecs.sum(axis=1)
train_test['counts_question1_tfidf'] = question1_tfidf_vecs.getnnz(axis=1)
train_test['mean_question1_tfidf'] = train_test['sum_question1_tfidf'] / train_test['counts_question1_tfidf']
train_test['sum_question2_tfidf'] = question2_tfidf_vecs.sum(axis=1)
train_test['counts_question2_tfidf'] = question2_tfidf_vecs.getnnz(axis=1)
train_test['mean_question2_tfidf'] = train_test['sum_question2_tfidf'] / train_test['counts_question2_tfidf']

tsvd = TruncatedSVD(n_components=200, random_state = 2016)
question1_tsvd_vecs = tsvd.fit_transform(question1_tfidf_vecs)
question2_tsvd_vecs = tsvd.fit_transform(question2_tfidf_vecs)

train_test['cosine_distance_tsvd'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_tsvd_vecs),
                                                          np.nan_to_num(question2_tsvd_vecs))]

train_test['cityblock_distance_tsvd'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_tsvd_vecs),
                                                          np.nan_to_num(question2_tsvd_vecs))]

train_test['jaccard_distance_tsvd'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_tsvd_vecs),
                                                          np.nan_to_num(question2_tsvd_vecs))]

train_test['canberra_distance_tsvd'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_tsvd_vecs),
                                                          np.nan_to_num(question2_tsvd_vecs))]

train_test['euclidean_distance_tsvd'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_tsvd_vecs),
                                                          np.nan_to_num(question2_tsvd_vecs))]

train_test['minkowski_distance_tsvd'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_tsvd_vecs),
                                                          np.nan_to_num(question2_tsvd_vecs))]

train_test['braycurtis_distance_tsvd'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_tsvd_vecs),
                                                          np.nan_to_num(question2_tsvd_vecs))]

train_test['skew_q1_tsvd'] = [skew(x) for x in np.nan_to_num(question1_tsvd_vecs)]
train_test['skew_q2_tsvd'] = [skew(x) for x in np.nan_to_num(question2_tsvd_vecs)]
train_test['kur_q1_tsvd'] = [kurtosis(x) for x in np.nan_to_num(question1_tsvd_vecs)]
train_test['kur_q2_tsvd'] = [kurtosis(x) for x in np.nan_to_num(question2_tsvd_vecs)]



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [38]:
train_test.head(5)

,id,question1,question2,test_id,question1_length,question2_length,length_diff,question1_word_length,question2_word_length,word_length_diff,len_char_question1,len_char_question2,common_words,ratio_question1,ratio_question2,cosine_similarity,question1_unigram,question2_unigram,question1_bigram,question2_bigram,question1_trigram,question2_trigram,question1_unigram_question2_unigram,question1_bigram_question2_unigram,count_of_question1_unigram,count_of_unique_question1_unigram,ratio_of_unique_question1_unigram,count_of_question1_bigram,count_of_unique_question1_bigram,ratio_of_unique_question1_bigram,count_of_question1_trigram,count_of_unique_question1_trigram,ratio_of_unique_question1_trigram,count_of_question2_unigram,count_of_unique_question2_unigram,ratio_of_unique_question2_unigram,count_of_question2_bigram,count_of_unique_question2_bigram,ratio_of_unique_question2_bigram,count_of_question2_trigram,count_of_unique_question2_trigram,ratio_of_unique_question2_trigram,count_of_question1_unigram_in_question2,ratio_of_question1_unigram_in_question2,count_of_question1_bigram_in_question2,ratio_of_question1_bigram_in_question2,count_of_question1_trigram_in_question2,ratio_of_question1_trigram_in_question2,pos_of_question1_unigram_in_question2_min,pos_of_question1_unigram_in_question2_mean,pos_of_question1_unigram_in_question2_median,pos_of_question1_unigram_in_question2_max,pos_of_question1_unigram_in_question2_std,normalized_pos_of_question1_unigram_in_question2_min,normalized_pos_of_question1_unigram_in_question2_mean,normalized_pos_of_question1_unigram_in_question2_median,normalized_pos_of_question1_unigram_in_question2_max,normalized_pos_of_question1_unigram_in_question2_std,pos_of_question1_bigram_in_question2_min,pos_of_question1_bigram_in_question2_mean,pos_of_question1_bigram_in_question2_median,pos_of_question1_bigram_in_question2_max,pos_of_question1_bigram_in_question2_std,normalized_pos_of_question1_bigram_in_question2_min,normalized_pos_of_question1_bigram_in_question2_mean,normalized_pos_of_question1_bigram_in_question2_median,normalized_pos_of_question1_bigram_in_question2_max,normalized_pos_of_question1_bigram_in_question2_std,pos_of_question1_trigram_in_question2_min,pos_of_question1_trigram_in_question2_mean,pos_of_question1_trigram_in_question2_median,pos_of_question1_trigram_in_question2_max,pos_of_question1_trigram_in_question2_std,normalized_pos_of_question1_trigram_in_question2_min,normalized_pos_of_question1_trigram_in_question2_mean,normalized_pos_of_question1_trigram_in_question2_median,normalized_pos_of_question1_trigram_in_question2_max,normalized_pos_of_question1_trigram_in_question2_std,jaccard_coef_of_unigram_between_question2_question1,jaccard_coef_of_bigram_between_question2_question1,jaccard_coef_of_trigram_between_question2_question1,dice_dist_of_unigram_between_question2_question1,dice_dist_of_bigram_between_question2_question1,dice_dist_of_trigram_between_question2_question1,fuzz_ratio,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio,wmd,cosine_distance_w2v,cityblock_distance_w2v,jaccard_distance_w2v,canberra_distance_w2v,euclidean_distance_w2v,minkowski_distance_w2v,braycurtis_distance_w2v,skew_q1_w2v,skew_q2_w2v,kur_q1_w2v,kur_q2_w2v,sum_question1_tfidf,counts_question1_tfidf,mean_question1_tfidf,sum_question2_tfidf,counts_question2_tfidf,mean_question2_tfidf,cosine_distance_tsvd,cityblock_distance_tsvd,jaccard_distance_tsvd,canberra_distance_tsvd,euclidean_distance_tsvd,minkowski_distance_tsvd,braycurtis_distance_tsvd,skew_q1_tsvd,skew_q2_tsvd,kur_q1_tsvd,kur_q2_tsvd
0,0.0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,NaN,66,57,9,14,12,2,20,20,13,0.928571,1.083333,0.944911,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...","[what_is, is_the, the_step, step_by, by_step, ...","[what_is, is_the, t